# Service Strain Index (SSI) Model
## UIDAI Datathon 2026 - Project ALI

This notebook demonstrates the mathematical formulation and practical application of the **Service Strain Index (SSI)** metric.

**Purpose:** Quantify operational strain in the Aadhaar ecosystem to prioritize resource allocation.

## 1. Introduction to SSI

### What is Service Strain Index?

The Service Strain Index (SSI) is a derived metric that measures the ratio of **demand** (demographic population requiring biometric updates) to **supply** (actual biometric updates processed).

### Mathematical Formulation:

$$
SSI = \frac{\text{Demographic Demand}}{\text{Biometric Supply} + 1}
$$

Where:
- **Demographic Demand**: Total population in the target age group (e.g., 5-17 years)
- **Biometric Supply**: Number of biometric updates completed
- **+1**: Prevents division by zero when supply is zero

### Interpretation:

| SSI Range | Status | Action Required |
|-----------|--------|----------------|
| SSI < 0.8 | Over-served | Monitor for data anomalies |
| 0.8 ≤ SSI ≤ 1.2 | Equilibrium | Maintain current operations |
| 1.2 < SSI ≤ 2.0 | Moderate Strain | Plan resource deployment |
| SSI > 2.0 | High Strain | **Immediate intervention needed** |

## 2. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add src directory to path
sys.path.insert(0, os.path.abspath('..'))

from src.data_loader import load_and_merge_data
from src.analytics import calculate_ssi

# Set visual style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.dpi'] = 100

print("Libraries imported successfully!")

In [ ]:
# Load merged dataset
df = load_and_merge_data(base_path="../data/raw")
print(f"Dataset loaded with shape: {df.shape}")

## 3. Step-by-Step SSI Calculation

Let's walk through the SSI calculation process with a sample district.

In [ ]:
# Select a sample district for demonstration
if 'district' in df.columns and len(df) > 0:
    sample_district = df.groupby('district').size().idxmax()  # District with most records
    sample_data = df[df['district'] == sample_district].head(5)
    
    print(f"Sample District: {sample_district}\n")
    print("Sample Data (first 5 records):")
    print(sample_data[['district', 'pincode', 'demo_age_5_17', 'bio_age_5_17']].to_string(index=False))
else:
    print("Insufficient data for demonstration")

In [ ]:
# Manual SSI calculation for understanding
if len(sample_data) > 0:
    print("\n=== Manual SSI Calculation ===")
    for idx, row in sample_data.iterrows():
        demand = row['demo_age_5_17']
        supply = row['bio_age_5_17']
        ssi = demand / (supply + 1)
        
        print(f"\nPincode: {row['pincode']}")
        print(f"  Demographic Demand: {demand}")
        print(f"  Biometric Supply: {supply}")
        print(f"  SSI = {demand} / ({supply} + 1) = {ssi:.3f}")
        
        if ssi > 2.0:
            print(f"  ⚠️  STATUS: High Strain - Immediate Action Required")
        elif ssi > 1.2:
            print(f"  ⚡ STATUS: Moderate Strain - Plan Deployment")
        else:
            print(f"  ✅ STATUS: Adequate Service")

## 4. Applying SSI to Entire Dataset

In [ ]:
# Calculate SSI for all records
df_with_ssi = calculate_ssi(df)

print("SSI Statistics:")
print(df_with_ssi['SSI'].describe())

# Categorize by strain level
df_with_ssi['strain_category'] = pd.cut(
    df_with_ssi['SSI'],
    bins=[0, 0.8, 1.2, 2.0, np.inf],
    labels=['Over-served', 'Equilibrium', 'Moderate Strain', 'High Strain']
)

print("\nDistribution by Strain Category:")
print(df_with_ssi['strain_category'].value_counts())

## 5. Visualizing SSI Distribution

In [ ]:
# Distribution plot
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df_with_ssi['SSI'], bins=50, color='#3498db', edgecolor='black', alpha=0.7)
axes[0].axvline(1.2, color='orange', linestyle='--', label='Moderate Strain Threshold')
axes[0].axvline(2.0, color='red', linestyle='--', label='High Strain Threshold')
axes[0].set_xlabel('SSI Score', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('SSI Distribution Across All Pincodes', fontsize=14)
axes[0].legend()
axes[0].set_xlim(0, min(df_with_ssi['SSI'].max(), 10))  # Cap for visualization

# Box plot by state (if applicable)
if 'state' in df_with_ssi.columns:
    top_states = df_with_ssi['state'].value_counts().head(10).index
    df_top_states = df_with_ssi[df_with_ssi['state'].isin(top_states)]
    
    df_top_states.boxplot(column='SSI', by='state', ax=axes[1], rot=45)
    axes[1].set_title('SSI Variation by Top 10 States', fontsize=14)
    axes[1].set_xlabel('State', fontsize=12)
    axes[1].set_ylabel('SSI Score', fontsize=12)
    axes[1].set_ylim(0, 5)  # Cap for better visualization
    plt.suptitle('')  # Remove auto-generated title

plt.tight_layout()
plt.show()

## 6. Priority District Identification

Using SSI to identify districts requiring immediate intervention.

In [ ]:
# Aggregate SSI by district
district_ssi = df_with_ssi.groupby('district').agg({
    'SSI': 'mean',
    'demo_age_5_17': 'sum',
    'bio_age_5_17': 'sum'
}).reset_index()

district_ssi = district_ssi.sort_values(by='SSI', ascending=False)

print("\n🚨 Top 10 Priority Districts (Highest SSI):")
print(district_ssi.head(10).to_string(index=False))

# Visualize top priority districts
plt.figure(figsize=(12, 6))
top_10 = district_ssi.head(10)
plt.barh(top_10['district'], top_10['SSI'], color='#e74c3c')
plt.xlabel('Average SSI Score', fontsize=12)
plt.ylabel('District', fontsize=12)
plt.title('Top 10 Priority Districts by SSI', fontsize=14)
plt.axvline(2.0, color='black', linestyle='--', label='High Strain Threshold')
plt.legend()
plt.tight_layout()
plt.show()

## 7. Actionable Insights

### Operational Recommendations:

Based on SSI analysis, we can generate specific action items:

In [ ]:
# Generate action items
high_strain_districts = district_ssi[district_ssi['SSI'] > 2.0]
moderate_strain_districts = district_ssi[(district_ssi['SSI'] > 1.2) & (district_ssi['SSI'] <= 2.0)]

print("\n📋 ACTIONABLE INSIGHTS:\n")
print(f"🔴 {len(high_strain_districts)} districts require IMMEDIATE mobile camp deployment")
print(f"🟡 {len(moderate_strain_districts)} districts need planned resource augmentation")

if len(high_strain_districts) > 0:
    total_backlog = high_strain_districts['demo_age_5_17'].sum() - high_strain_districts['bio_age_5_17'].sum()
    print(f"\n📊 Estimated Backlog in High-Strain Districts: {total_backlog:,.0f} updates pending")
    
    avg_daily_capacity = 500  # Assume 500 updates per mobile unit per day
    units_needed = np.ceil(total_backlog / (avg_daily_capacity * 30))  # 30-day deployment
    print(f"💡 Recommended Mobile Units: {int(units_needed)} (for 30-day clearance)")

## 8. Model Validation

### Why SSI Works:

1. **Simplicity**: Easy to calculate and interpret
2. **Actionable**: Direct mapping to resource allocation decisions
3. **Scalable**: Can be applied at pincode, district, or state level
4. **Real-time Ready**: Can be updated with streaming data

### Limitations:

- Assumes linear relationship between demand and supply
- Does not account for temporal trends (seasonal variations)
- May not capture quality of service, only quantity

### Future Enhancements:

- **Time-weighted SSI**: Give more weight to recent data
- **Composite SSI**: Include authentication success rates
- **Predictive SSI**: Use ML to forecast future strain

---
## Conclusion

The Service Strain Index (SSI) provides a quantitative, data-driven approach to prioritizing Aadhaar operational resources. By systematically identifying high-strain zones, UIDAI can optimize mobile camp deployments and improve service coverage across India.

**Project ALI (Aadhaar Lifecycle Intelligence)**  
UIDAI Datathon 2026  
Prepared by: Priyanshu